In [4]:
# -*- coding:utf-8 -*-
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from data.cifar import CIFAR10, CIFAR100
from data.mnist import MNIST
from model import CNN
import argparse, sys
import numpy as np
import datetime
import shutil

In [5]:
def train(args, model, device, train_loader, optimizer, epoch, eps=9.9):
    model.train()
   # for batch_idx, (data, target, idx, is_pure, is_corrupt) in enumerate(train_loader):
    for batch_idx, (data, target,index) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = F.softmax(output, dim=1)
        output = output + (output[:, 10] / eps).unsqueeze(1)
        output.log_()
        
        loss = F.nll_loss(output, target)
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    acc = []
    with torch.no_grad():
        for data, target, index in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output[:,:10].argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc=100. * correct / len(test_loader.dataset);
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        acc))
    return acc

In [10]:
def main():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    
    parser.add_argument('--result_dir', type = str, help = 'dir to save result txt files', default = 'results/')
    parser.add_argument('--noise_rate', type = float, help = 'corruption rate, should be less than 1', default = 0.5)
    parser.add_argument('--forget_rate', type = float, help = 'forget rate', default = None)
    parser.add_argument('--noise_type', type = str, help='[pairflip, symmetric]', default='symmetric')
    parser.add_argument('--num_gradual', type = int, default = 10, help='how many epochs for linear drop rate, can be 5, 10, 15. This parameter is equal to Tk for R(T) in Co-teaching paper.')
    parser.add_argument('--exponent', type = float, default = 1, help='exponent of the forget rate, can be 0.5, 1, 2. This parameter is equal to c in Tc for R(T) in Co-teaching paper.')
    parser.add_argument('--top_bn', action='store_true')
    parser.add_argument('--dataset', type = str, help = 'mnist, cifar10, or cifar100', default = 'mnist')
    parser.add_argument('--n_epoch', type=int, default=200)
    parser.add_argument('--seed', type=int, default=1)
    parser.add_argument('--print_freq', type=int, default=50)
    parser.add_argument('--num_workers', type=int, default=4, help='how many subprocesses to use for data loading')
    parser.add_argument('--num_iter_per_epoch', type=int, default=400)
    parser.add_argument('--epoch_decay_start', type=int, default=80)

    
    parser.add_argument('--batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for training (default: 256)')
    parser.add_argument('--test-batch-size', type=int, default=4000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--lr', type=float, default=0.005, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')
    
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    
    batch_size=args.batch_size
    
    if args.dataset=='mnist':
        input_channel=1
        num_classes=10
        args.top_bn = False
        args.epoch_decay_start = 80
        args.n_epoch = 200
        train_dataset = MNIST(root='./data/',
                                    download=True,  
                                    train=True, 
                                    transform=transforms.ToTensor(),
                                    noise_type=args.noise_type,
                                    noise_rate=args.noise_rate
                             )

        test_dataset = MNIST(root='./data/',
                                   download=True,  
                                   train=False, 
                                   transform=transforms.ToTensor(),
                                   noise_type=args.noise_type,
                                   noise_rate=args.noise_rate
                            )
    
    if args.dataset=='cifar10':
        input_channel=3
        num_classes=10
        args.top_bn = False
        args.epoch_decay_start = 80
        args.n_epoch = 200
        train_dataset = CIFAR10(root='./data/',
                                    download=True,  
                                    train=True, 
                                    transform=transforms.ToTensor(),
                                    noise_type=args.noise_type,
                                    noise_rate=args.noise_rate
                               )

        test_dataset = CIFAR10(root='./data/',
                                    download=True,  
                                    train=False, 
                                    transform=transforms.ToTensor(),
                                    noise_type=args.noise_type,
                                    noise_rate=args.noise_rate
                              )

    if args.dataset=='cifar100':
        input_channel=3
        num_classes=100
        args.top_bn = False
        args.epoch_decay_start = 100
        args.n_epoch = 200
        train_dataset = CIFAR100(root='./data/',
                                    download=True,  
                                    train=True, 
                                    transform=transforms.ToTensor(),
                                    noise_type=args.noise_type,
                                    noise_rate=args.noise_rate
                                )

        test_dataset = CIFAR100(root='./data/',
                                    download=True,  
                                    train=False, 
                                    transform=transforms.ToTensor(),
                                    noise_type=args.noise_type,
                                    noise_rate=args.noise_rate
                                )

    if args.forget_rate is None:
        forget_rate=args.noise_rate
    else:
        forget_rate=args.forget_rate

    noise_or_not = train_dataset.noise_or_not
    # Data Loader (Input Pipeline)
    print('loading dataset...')
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size, 
                                               num_workers=args.num_workers,
                                               drop_last=True,
                                               shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size, 
                                              num_workers=args.num_workers,
                                              drop_last=True,
                                              shuffle=False)
    # Define models
    print('building model...')
    cnn1 = CNN(input_channel=input_channel, n_outputs=num_classes+1)
    cnn1.cuda()
    print(cnn1.parameters)
    #optimizer1 = torch.optim.SGD(cnn1.parameters(), lr=learning_rate)
    optimizer = torch.optim.SGD(cnn1.parameters(), lr=args.lr, momentum=args.momentum)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    acc=[]
    loss=[]
    loss_pure=[]
    loss_corrupt=[]
    for epoch in range(1, args.n_epoch + 1):
        if epoch<20:
            l1=train(args, cnn1, device, train_loader, optimizer, epoch,eps=9.7)
            loss.append(l1)
            acc.append(test(args, cnn1, device, test_loader))
        else:
            l1=train(args, cnn1, device, train_loader, optimizer, epoch,eps=1.0)
            loss.append(l1)
            acc.append(test(args, cnn1, device, test_loader))
        #loss.append(train(args, model, device, train_loader, optimizer, epoch, eps=9.3))
        #l1=train(args, cnn1, device, train_loader, optimizer, epoch,eps=4.9)
        #loss.append(l1)
        
        #acc.append(test(args, cnn1, device, test_loader))
    name=str(args.dataset)+" "+str(args.noise_type)+" "+str(args.noise_rate)
    #np.save(name+"acc.npy",acc)
    #np.save(name+"loss.npy",loss)
   

In [14]:
torch.cuda.set_device(0)#eps=9.1
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -2.8985, Accuracy: 7024/10000 (70%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 1.165180


KeyboardInterrupt: 

In [16]:
torch.cuda.set_device(0)#eps=8.5
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0

KeyboardInterrupt: 

In [4]:
torch.cuda.set_device(0)#eps=8.0
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -4.2712, Accuracy: 9675/10000 (97%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 1.593991

Test set: Average loss: -4.2877, Accuracy: 9683/10000 (97%)

Train Epoch: 53 [0/60000 (0%)]	Loss: 1.553949

Test set: Average loss: -4.0296, Accuracy: 9578/10000 (96%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 1.457996

Test set: Average loss: -4.1761, Accuracy: 9590/10000 (96%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 1.465689

Test set: Average loss: -4.0873, Accuracy: 9496/10000 (95%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 1.510102

Test set: Average loss: -4.1359, Accuracy: 9466/10000 (95%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 1.475302

Test set: Average loss: -4.1878, Accuracy: 9500/10000 (95%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 1.430648

Test set: Average loss: -4.0842, Accuracy: 9394/10000 (94%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 1.424664

Test set: Average loss: -3.9793, Accuracy: 9286/10000 (93%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 1.444014

Test set: Average loss: -4.

KeyboardInterrupt: 

In [6]:
torch.cuda.set_device(0)#eps=7.5
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", 

In [14]:
torch.cuda.set_device(1)#eps=7.5
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.45
[[0.55 0.45 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.55 0.45 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.55 0.45 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.55 0.45 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.55 0.45 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.55 0.45 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.55 0.45 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.55 0.45 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.55 0.45]
 [0.45 0.   0.   0.   0.   0.   0.   0.   0.   0.55]]
loading dataset...
building model...
<bound method Module.parameters of CNN(
  (c1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p


Test set: Average loss: -6.8369, Accuracy: 5531/10000 (55%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 0.434584

Test set: Average loss: -6.6614, Accuracy: 5944/10000 (59%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 0.421429

Test set: Average loss: -6.9249, Accuracy: 6083/10000 (61%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 0.434849

Test set: Average loss: -7.0216, Accuracy: 6067/10000 (61%)

Train Epoch: 61 [0/60000 (0%)]	Loss: 0.393799

Test set: Average loss: -6.9364, Accuracy: 5588/10000 (56%)

Train Epoch: 62 [0/60000 (0%)]	Loss: 0.362103

Test set: Average loss: -7.0267, Accuracy: 5567/10000 (56%)

Train Epoch: 63 [0/60000 (0%)]	Loss: 0.386716

Test set: Average loss: -6.9433, Accuracy: 6128/10000 (61%)

Train Epoch: 64 [0/60000 (0%)]	Loss: 0.379187

Test set: Average loss: -7.0836, Accuracy: 5727/10000 (57%)

Train Epoch: 65 [0/60000 (0%)]	Loss: 0.368382

Test set: Average loss: -7.1219, Accuracy: 5601/10000 (56%)

Train Epoch: 66 [0/60000 (0%)]	Loss: 0.378955

Test set: Average loss: -7.

Train Epoch: 133 [0/60000 (0%)]	Loss: 0.046991

Test set: Average loss: -8.7996, Accuracy: 5780/10000 (58%)

Train Epoch: 134 [0/60000 (0%)]	Loss: 0.060962

Test set: Average loss: -8.7968, Accuracy: 5649/10000 (56%)

Train Epoch: 135 [0/60000 (0%)]	Loss: 0.070663

Test set: Average loss: -8.7074, Accuracy: 5545/10000 (55%)

Train Epoch: 136 [0/60000 (0%)]	Loss: 0.065952

Test set: Average loss: -8.7572, Accuracy: 5621/10000 (56%)

Train Epoch: 137 [0/60000 (0%)]	Loss: 0.044334

Test set: Average loss: -8.5475, Accuracy: 5590/10000 (56%)

Train Epoch: 138 [0/60000 (0%)]	Loss: 0.050648

Test set: Average loss: -8.5385, Accuracy: 5371/10000 (54%)

Train Epoch: 139 [0/60000 (0%)]	Loss: 0.062324

Test set: Average loss: -9.0247, Accuracy: 5673/10000 (57%)

Train Epoch: 140 [0/60000 (0%)]	Loss: 0.043791

Test set: Average loss: -8.6967, Accuracy: 5535/10000 (55%)

Train Epoch: 141 [0/60000 (0%)]	Loss: 0.041003

Test set: Average loss: -8.6515, Accuracy: 5431/10000 (54%)

Train Epoch: 142 [0

In [6]:
torch.cuda.set_device(1)#eps=4.7symmetric
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -5.3040, Accuracy: 9849/10000 (98%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 1.291684

Test set: Average loss: -5.1820, Accuracy: 9852/10000 (99%)

Train Epoch: 53 [0/60000 (0%)]	Loss: 1.284720

Test set: Average loss: -5.1434, Accuracy: 9826/10000 (98%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 1.222273

Test set: Average loss: -5.1434, Accuracy: 9826/10000 (98%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 1.226403

Test set: Average loss: -5.1347, Accuracy: 9815/10000 (98%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 1.255898

Test set: Average loss: -5.0695, Accuracy: 9778/10000 (98%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 1.247830

Test set: Average loss: -5.2765, Accuracy: 9809/10000 (98%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 1.217032

Test set: Average loss: -5.1461, Accuracy: 9783/10000 (98%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 1.177559

Test set: Average loss: -5.0968, Accuracy: 9796/10000 (98%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 1.190421

Test set: Average loss: -5.

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._

KeyboardInterrupt: 

In [10]:
torch.cuda.set_device(1)#eps=4.3 symmetric
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -5.2839, Accuracy: 9851/10000 (99%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 1.247290

Test set: Average loss: -5.1560, Accuracy: 9833/10000 (98%)

Train Epoch: 53 [0/60000 (0%)]	Loss: 1.228469

Test set: Average loss: -5.0340, Accuracy: 9825/10000 (98%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 1.172806

Test set: Average loss: -5.1717, Accuracy: 9827/10000 (98%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 1.188939

Test set: Average loss: -5.1100, Accuracy: 9832/10000 (98%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 1.208518

Test set: Average loss: -5.0183, Accuracy: 9798/10000 (98%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 1.186064

Test set: Average loss: -5.4184, Accuracy: 9834/10000 (98%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 1.168273

Test set: Average loss: -5.2457, Accuracy: 9809/10000 (98%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 1.135994

Test set: Average loss: -5.3137, Accuracy: 9810/10000 (98%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 1.165443

Test set: Average loss: -5.


Test set: Average loss: -6.4223, Accuracy: 9043/10000 (90%)

Train Epoch: 128 [0/60000 (0%)]	Loss: 0.804618

Test set: Average loss: -6.4355, Accuracy: 9130/10000 (91%)

Train Epoch: 129 [0/60000 (0%)]	Loss: 0.824793

Test set: Average loss: -6.7116, Accuracy: 9127/10000 (91%)

Train Epoch: 130 [0/60000 (0%)]	Loss: 0.795532

Test set: Average loss: -6.4030, Accuracy: 9022/10000 (90%)

Train Epoch: 131 [0/60000 (0%)]	Loss: 0.771030

Test set: Average loss: -6.6163, Accuracy: 9128/10000 (91%)

Train Epoch: 132 [0/60000 (0%)]	Loss: 0.818227

Test set: Average loss: -6.2911, Accuracy: 9017/10000 (90%)

Train Epoch: 133 [0/60000 (0%)]	Loss: 0.773180

Test set: Average loss: -6.8140, Accuracy: 9137/10000 (91%)

Train Epoch: 134 [0/60000 (0%)]	Loss: 0.789660

Test set: Average loss: -6.5502, Accuracy: 9079/10000 (91%)

Train Epoch: 135 [0/60000 (0%)]	Loss: 0.738685

Test set: Average loss: -6.3943, Accuracy: 9008/10000 (90%)

Train Epoch: 136 [0/60000 (0%)]	Loss: 0.770635

Test set: Average 

Traceback (most recent call last):
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ziyinl/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._

KeyboardInterrupt: 

In [ ]:
torch.cuda.set_device(1)　#eps=4.9 symmetric
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -5.2602, Accuracy: 9845/10000 (98%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 1.335634

Test set: Average loss: -5.0349, Accuracy: 9819/10000 (98%)

Train Epoch: 53 [0/60000 (0%)]	Loss: 1.302231

Test set: Average loss: -4.9712, Accuracy: 9829/10000 (98%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 1.234406

Test set: Average loss: -5.1395, Accuracy: 9815/10000 (98%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 1.262694

Test set: Average loss: -5.2710, Accuracy: 9823/10000 (98%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 1.280034

Test set: Average loss: -5.1497, Accuracy: 9777/10000 (98%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 1.250578

Test set: Average loss: -5.3573, Accuracy: 9810/10000 (98%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 1.235952

Test set: Average loss: -5.1230, Accuracy: 9789/10000 (98%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 1.220565

Test set: Average loss: -5.2887, Accuracy: 9792/10000 (98%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 1.228614

Test set: Average loss: -5.

In [11]:
torch.cuda.set_device(1)　#eps=4.9 symmetric
sys.argv = ['-f']
main()

9 10
60000
Actual noise 0.50
[[0.5        0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.5        0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.5        0.05555556 0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.5        0.05555556 0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.5        0.05555556
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.5
  0.05555556 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.5        0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
  0.05555556 0.5        0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0


Test set: Average loss: -0.0454, Accuracy: 9646/10000 (96%)

Train Epoch: 52 [0/60000 (0%)]	Loss: 0.000664

Test set: Average loss: -0.0407, Accuracy: 9646/10000 (96%)

Train Epoch: 53 [0/60000 (0%)]	Loss: 0.000637

Test set: Average loss: -0.0383, Accuracy: 9667/10000 (97%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 0.000607

Test set: Average loss: -0.0346, Accuracy: 9642/10000 (96%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 0.000573

Test set: Average loss: -0.0330, Accuracy: 9669/10000 (97%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 0.000583

Test set: Average loss: -0.0289, Accuracy: 9659/10000 (97%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 0.000560

Test set: Average loss: -0.0266, Accuracy: 9666/10000 (97%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 0.000537

Test set: Average loss: -0.0239, Accuracy: 9659/10000 (97%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 0.000521

Test set: Average loss: -0.0191, Accuracy: 9624/10000 (96%)

Train Epoch: 60 [0/60000 (0%)]	Loss: 0.000494

Test set: Average loss: -0.

Train Epoch: 128 [0/60000 (0%)]	Loss: 0.000178

Test set: Average loss: 0.0933, Accuracy: 9656/10000 (97%)

Train Epoch: 129 [0/60000 (0%)]	Loss: 0.000173

Test set: Average loss: 0.0919, Accuracy: 9647/10000 (96%)

Train Epoch: 130 [0/60000 (0%)]	Loss: 0.000170

Test set: Average loss: 0.0941, Accuracy: 9652/10000 (97%)

Train Epoch: 131 [0/60000 (0%)]	Loss: 0.000166

Test set: Average loss: 0.0950, Accuracy: 9636/10000 (96%)

Train Epoch: 132 [0/60000 (0%)]	Loss: 0.000166

Test set: Average loss: 0.0945, Accuracy: 9665/10000 (97%)

Train Epoch: 133 [0/60000 (0%)]	Loss: 0.000169

Test set: Average loss: 0.0955, Accuracy: 9669/10000 (97%)

Train Epoch: 134 [0/60000 (0%)]	Loss: 0.000161

Test set: Average loss: 0.0984, Accuracy: 9636/10000 (96%)

Train Epoch: 135 [0/60000 (0%)]	Loss: 0.000172

Test set: Average loss: 0.0979, Accuracy: 9653/10000 (97%)

Train Epoch: 136 [0/60000 (0%)]	Loss: 0.000164

Test set: Average loss: 0.0994, Accuracy: 9654/10000 (97%)

Train Epoch: 137 [0/60000 (0